<a href="https://colab.research.google.com/github/parapara29/Tweet-Classification-using-LLM/blob/main/zephyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the Data and Datasets**

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling

In [ ]:
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import torch
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
from datasets import Dataset
import warnings
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_excel('/content/drive/MyDrive/INTERIIT/behaviour_simulation_train.xlsx', usecols=[1,2,3,4,6])
df.head()

,date,likes,content,username,inferred company
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,tim hortons
1,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,independent
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,cbc
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,williams
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,independent


In [ ]:
captions = pd.read_csv('/content/drive/MyDrive/INTERIIT/captions (2).csv')
captions.head()

,Captions
0,Picture of Tim Hortons
1,Picture of the rapper is in the studio
2,Picture of A soldier in afghanistan
3,"Picture of The beach in the town of sicily, italy"
4,Picture of a student receiving a certificate f...


In [ ]:
df_new = pd.concat([df, captions], axis=1)

In [ ]:
df_new = df_new[:10000]

In [ ]:
df_new.isna().sum()

date                0
likes               0
content             0
username            0
inferred company    0
Captions            0
dtype: int64

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
df_new

,date,likes,content,username,inferred company,Captions
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,tim hortons,Picture of Tim Hortons
1,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,independent,Picture of the rapper is in the studio
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,cbc,Picture of A soldier in afghanistan
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,williams,"Picture of The beach in the town of sicily, italy"
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,independent,Picture of a student receiving a certificate f...
...,...,...,...,...,...,...
9995,2019-08-18 15:00:02,5,Just hangin' out in our #SundayBest with <hype...,lanebryant,lane bryant,Picture of the plus size bra I'm wearing in th...
9996,2019-05-29 21:55:06,2,"Taking on student loans can feel overwhelming,...",CharlesSchwab,charles schwab,Picture of A man hanging from a balloon
9997,2020-02-03 15:42:09,75,Chatting with former GM <mention> on how the C...,PatMcAfeeShow,mcafee,Picture of the man in the black shirt and the ...
9998,2020-01-01 04:10:33,0,Let’s Begin New Year With Renewal Of Hope – Bo...,IndependentNGR,independent,Picture of The president of the republic of ni...


In [ ]:
df_new['text'] = "Date: " + df_new['date'].astype(str) + ", Username: " + df_new['username'].astype(str) + ", Inferred company: " + df_new['inferred company'].astype(str) + ", Content: " + df_new['content'].astype(str) + ", Image Description: " + df_new['Captions'].astype(str)

In [ ]:
columns_to_drop = ['date', 'username','inferred company', 'content', 'Captions',]
df_new.drop(columns=columns_to_drop, inplace=True)
df_new

,likes,text
0,1,"Date: 2020-12-12 00:47:00, Username: TimHorton..."
1,2750,"Date: 2018-06-30 10:04:20, Username: IndyMusic..."
2,57,"Date: 2020-09-29 19:47:28, Username: CBCCanada..."
3,152,"Date: 2020-10-01 11:40:09, Username: MKWilliam..."
4,41,"Date: 2018-10-19 14:30:46, Username: BGISD, In..."
...,...,...
9995,5,"Date: 2019-08-18 15:00:02, Username: lanebryan..."
9996,2,"Date: 2019-05-29 21:55:06, Username: CharlesSc..."
9997,75,"Date: 2020-02-03 15:42:09, Username: PatMcAfee..."
9998,0,"Date: 2020-01-01 04:10:33, Username: Independe..."


In [ ]:
df_new.reset_index(drop=True, inplace=True)

In [ ]:
df_new['likes']=df_new['likes'].astype('str')

In [ ]:
df_new.isna().sum()

likes    0
text     0
dtype: int64

In [ ]:
dataset = Dataset.from_pandas(df_new, split='train')
dataset

Dataset({
    features: ['likes', 'text'],
    num_rows: 10000
})

In [ ]:
MODEL_CKPT = 'allenai/longformer-base-4096'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = AutoModelForMaskedLM.from_pretrained(MODEL_CKPT).to(device)

cuda


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

In [ ]:
def tokenize_function(batched_data):
    # Assuming 'text' and 'likes' are your column names in the dataset
    result = tokenizer(batched_data['text'], padding='max_length', truncation=True, max_length=1024)
    # Add the 'likes' column to the tokenized output
    result['likes'] = batched_data['likes']
    if tokenizer.is_fast:
        result['word_ids'] = [result.word_ids(i) for i in range(len(result['input_ids']))]
    return result

# Assuming 'dataset' contains your concatenated CSV data
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# tokenized_datasets[0]

Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

In [ ]:
def chunk_and_tokenize(batched_data):
    tokenized_texts = []
    chunked_likes = []

    for text, likes in zip(batched_data['text'], batched_data['likes']):
        # Split the text into chunks of 1024 tokens
        text_chunks = [text[i:i + 1024] for i in range(0, len(text), 1024)]
        for chunk in text_chunks:
            # Tokenize each chunk
            tokenized_chunk = tokenizer(chunk, padding='max_length', truncation=True, max_length=1024)
            tokenized_texts.append(tokenized_chunk)
            chunked_likes.append(likes)  # Append corresponding likes for each chunk

    # Convert the lists of tokenized texts and chunked likes to datasets
    return {'input_ids': tokenized_texts, 'likes': chunked_likes}

# Assuming 'dataset' contains your concatenated CSV data
chunked_tokenized_datasets = dataset.map(chunk_and_tokenize, batched=True)

Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

In [ ]:
dataset_split = tokenized_datasets.train_test_split(test_size=0.1)
dataset_split

DatasetDict({
    train: Dataset({
        features: ['likes', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1774
    })
    test: Dataset({
        features: ['likes', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 198
    })
})

# **Zephyr**

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
!pip install peft trl

In [ ]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer,GPTQConfig, TrainingArguments
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
def process_data_sample(example):

        '''
        Helper function to process the dataset sample by adding prompt and clean if necessary.

        Args:
        example: Data sample

        Returns:
        processed_example: Data sample post processing
        '''

        processed_example = "<|system|>\n You are a prediction chatbot who helps with predicting the amount of likes a tweet will get, based on the date, content, image description, username, and inferred company in a professional manner.\n<|user|>\n" + example['text'] + "\n<|assistant|>\n" + example['likes']

        return processed_example

In [ ]:
df2 = pd.DataFrame()
df2['text'] = df_new[['text', 'likes']].apply(lambda x: process_data_sample(x), axis=1)
df2.head()
print(df2.iloc[5]['text'])

<|system|>
 You are a prediction chatbot who helps with predicting the amount of likes a tweet will get, based on the date, content, image description, username, and inferred company in a professional manner.
<|user|>
Date: 2020-11-15 16:01:08, Username: cnni, Inferred company: cnn, Content: An 85-year-old primary school in Shanghai has been lifted off the ground — in its entirety — and relocated using new technology dubbed the "walking machine." <hyperlink> <hyperlink>, Image Description: Picture of a concrete tower in the middle of a city
<|assistant|>
525


In [ ]:
processed_data = Dataset.from_pandas(df2[['text']])
processed_data

Dataset({
    features: ['text'],
    num_rows: 10000
})

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
new_model = "zephyr_fine_tune"
trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored

In [ ]:
!nvidia-smi

Tue Dec 12 06:02:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    28W /  70W |  14961MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
del model
del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

def process_data_sample(example):

    processed_example = "<|system|>\n You are a prediction chatbot who helps with predicting the amount of likes a tweet will get in a professional manner.\n<|user|>\n" + example['text'] + "\n<|assistant|>\n"

    return processed_example

In [ ]:
df['text'] = "Date: " + df['date'].astype(str) + ", Username: " + df['username'].astype(str) + ", Inferred company: " + df['inferred company'].astype(str) + ", Content: " + df['content'].astype(str) + ", Image Description: "

In [ ]:
pd.set_option('display.max_colwidth', None)
df[:10000]

,date,likes,content,username,inferred company,text
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>",TimHortonsPH,tim hortons,"Date: 2020-12-12 00:47:00, Username: TimHortonsPH, Inferred company: tim hortons, Content: Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>, Image Description:"
1,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>,IndyMusic,independent,"Date: 2018-06-30 10:04:20, Username: IndyMusic, Inferred company: independent, Content: Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>, Image Description:"
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>,CBCCanada,cbc,"Date: 2020-09-29 19:47:28, Username: CBCCanada, Inferred company: cbc, Content: Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>, Image Description:"
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, Italy redoubled its efforts, and they're now paying off. My latest. 👇🏼👇🏼\n<hyperlink> <hyperlink>",MKWilliamsRome,williams,"Date: 2020-10-01 11:40:09, Username: MKWilliamsRome, Inferred company: williams, Content: 1st in Europe to be devastated by COVID-19, Italy redoubled its efforts, and they're now paying off. My latest. 👇🏼👇🏼\n<hyperlink> <hyperlink>, Image Description:"
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>! She was presented a Beyond the Call Award for her leadership and planning last week’s Cultural Diversity Night! #excellenceisworththeeffort <hyperlink>,BGISD,independent,"Date: 2018-10-19 14:30:46, Username: BGISD, Inferred company: independent, Content: Congratulations to Pauletha Butts of <mention>! She was presented a Beyond the Call Award for her leadership and planning last week’s Cultural Diversity Night! #excellenceisworththeeffort <hyperlink>, Image Description:"
...,...,...,...,...,...,...
9995,2019-08-18 15:00:02,5,Just hangin' out in our #SundayBest with <hyperlink> 🌴\nShop Cacique: <hyperlink> <hyperlink>,lanebryant,lane bryant,"Date: 2019-08-18 15:00:02, Username: lanebryant, Inferred company: lane bryant, Content: Just hangin' out in our #SundayBest with <hyperlink> 🌴\nShop Cacique: <hyperlink> <hyperlink>, Image Description:"
9996,2019-05-29 21:55:06,2,"Taking on student loans can feel overwhelming, but <mention> has some ideas on how you can manage that debt so it works in your favor: <hyperlink> <hyperlink>",CharlesSchwab,charles schwab,"Date: 2019-05-29 21:55:06, Username: CharlesSchwab, Inferred company: charles schwab, Content: Taking on student loans can feel overwhelming, but <mention> has some ideas on how you can manage that debt so it works in your favor: <hyperlink> <hyperlink>, Image Description:"
9997,2020-02-03 15:42:09,75,Chatting with former GM <mention> on how the CBA negotiations will affect this upcoming offseason and some potential big names resigning with their respective teams #PatMcAfeeShowLIVE <hyperlink>,PatMcAfeeShow,mcafee,"Date: 2020-02-03 15:42:09, Username: PatMcAfeeShow, Inferred company: mcafee, Content: Chatting with former GM <mention> on how the CBA negotiations will affect this upcoming offseason and some potential big names resigning with their respective teams #PatMcAfeeShowLIVE <hyperlink>, Image Description:"
9998,2020-01-01 04:10:33,0,Let’s Begin New Year With Renewal Of Hope – Bode George <hyperlink> <hyperlink>,IndependentNGR,independent,"Date: 2020-01-01 04:10:33, Username: IndependentNGR, Inferred company: independent, Content: Let’s Begin New Year With Renewal Of Hope – Bode George <hyperlink> <hyperlink>, Image Description:"


In [ ]:
inp_str = process_data_sample(
    {
        "text": "Date: 2019-05-29 21:55:06, Username: CharlesSchwab, Inferred company: charles schwab, Content: Taking on student loans can feel overwhelming, but <mention> has some ideas on how you can manage that debt so it works in your favor: <hyperlink> <hyperlink>",
    }
)
inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a prediction chatbot who helps with predicting the amount of likes a tweet will get in a professional manner.
<|user|>
Date: 2019-05-29 21:55:06, Username: CharlesSchwab, Inferred company: charles schwab, Content: Taking on student loans can feel overwhelming, but <mention> has some ideas on how you can manage that debt so it works in your favor: <hyperlink> <hyperlink>
<|assistant|>
1016740000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
6.923364162445068
